In [6]:
import pandas as pd
import numpy as np
from datetime import date

dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   
df = pd.read_csv('../../data/train_test_data.csv')

df['month'] = df.Timestamp.str[5:7]
month2label = {j:i for i,j in enumerate(df['month'].unique())}
df['month'] = df['month'].map(month2label)
#train_df['location_country'].map(loc_country2idx)
h = list(df.Timestamp.str[11:13])
df['hour'] = h
hour2label = {j:i for i,j in enumerate(df['hour'].unique())}
df['hour'] = df['hour'].map(hour2label)

yy = df.Timestamp.str[:4]
mm = df.Timestamp.str[5:7]
dd = df.Timestamp.str[8:10]

y = [int(y) for y in list(yy)]
m = [int(m) for m in list(mm)]
d = [int(d) for d in list(dd)]
w = [date(i,j,k).weekday() for i,j,k in zip(y,m,d)]
df['week'] = w
w_h = [str(i)+'_'+j for i, j in zip(w, h)]
df['week_hour'] = w_h
week_hour2label = {j:i for i,j in enumerate(df['week_hour'].unique())}
df['week_hour'] = df['week_hour'].map(week_hour2label)


df.to_csv('../../data/train.csv', index=False)

df = pd.read_csv('../../data/train.csv', dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
diff = diff.fillna(pd.Timedelta(seconds=0))
diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

df['elapsed'] = diff

df["elapsed"] = df["elapsed"].apply(
    lambda x: x if x < 600 and x >= 0 else 0)

def elap(x):
    if x == 0:
        return 0
    elif x < 9:
        return 1
    elif x < 101:
        return 2
    elif x < 201:
        return 3
    elif x < 301:
        return 4
    elif x < 401:
        return 5
    elif x < 501:
        return 6
    elif x < 601:
        return 7
    
df.elapsed = df.elapsed.apply(lambda x : elap(x))

In [7]:
import time
from datetime import datetime
df.to_csv('../../data/train.csv', index=False)

df = pd.read_csv('../../data/train.csv')
def convert_time(s):
    timestamp = time.mktime(
        datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
    )
    return int(timestamp)

df['Timestamp'] = df.Timestamp.apply(convert_time)

In [8]:
KnowledgeTag = {j:i for i,j in enumerate(df['KnowledgeTag'].unique())}
df['KnowledgeTag'] = df['KnowledgeTag'].map(KnowledgeTag)

In [9]:
df['assessmentItemID0'] = df['assessmentItemID'].str[2]
assessmentItemID0 = {j:i for i,j in enumerate(df['assessmentItemID0'].unique())}
df['assessmentItemID0'] = df['assessmentItemID0'].map(assessmentItemID0)

df['assessmentItemID1'] = df['assessmentItemID'].str[4:7]
assessmentItemID1 = {j:i for i,j in enumerate(df['assessmentItemID1'].unique())}
df['assessmentItemID1'] = df['assessmentItemID1'].map(assessmentItemID1)

df['assessmentItemID2'] = df['assessmentItemID'].str[8:10]
assessmentItemID2 = {j:i for i,j in enumerate(df['assessmentItemID2'].unique())}
df['assessmentItemID2'] = df['assessmentItemID2'].map(assessmentItemID2)

df['testId0'] = df['testId'].str[2]
testId0 = {j:i for i,j in enumerate(df['testId0'].unique())}
df['testId0'] = df['testId0'].map(testId0)

df['testId1'] = df['testId'].str[7:10]
testId1 = {j:i for i,j in enumerate(df['testId1'].unique())}
df['testId1'] = df['testId1'].map(testId1)

df = df.drop(['assessmentItemID', 'testId'], axis=1)

def __augmentation(df):
    cnt = pd.DataFrame(df.groupby('userID')['answerCode'].count())
    cnt.answerCode = cnt.answerCode//100
    i_cnt = cnt[cnt['answerCode'] >= 1]

    n_index = -1
    for i,j in enumerate(i_cnt.index):
        c = i_cnt.iloc[i].answerCode
        index = df[df['userID'] == j].index
        for k in range(2, c+2):
            ii = index[-k*100:-100*(k-1)]
            if len(ii) > 20:
                df.loc[ii, ['userID']] = n_index
                n_index -= 1
    return df
df = __augmentation(df)

df.to_csv('../../data/train_data_3.0.csv', index=False)

In [18]:
train.assessmentItemID

0          A060001001
1          A060001002
2          A060001003
3          A060001004
4          A060001005
              ...    
2525951    A040197006
2525952    A040130001
2525953    A040130002
2525954    A040130003
2525955    A040130004
Name: assessmentItemID, Length: 2525956, dtype: object

In [12]:
train = pd.read_csv('../../data/train_test_data.csv')
df['assessmentItemID'] = train['assessmentItemID']
df['testId'] = train['testId']

testId = {j:i for i,j in enumerate(df['testId'].unique())}
assessmentItemID = {j:i for i,j in enumerate(df['assessmentItemID'].unique())}

df['testId'] = df['testId'].map(testId)
df['assessmentItemID'] = df['assessmentItemID'].map(assessmentItemID)
df

,userID,answerCode,Timestamp,KnowledgeTag,month,hour,week,week_hour,elapsed,assessmentItemID0,assessmentItemID1,assessmentItemID2,testId0,testId1,assessmentItemID,testId
0,-7,1,1585009031,0,0,0,1,0,0,0,0,0,0,0,0,0
1,-7,1,1585009034,1,0,0,1,0,1,0,0,1,0,0,1,0
2,-7,1,1585009042,1,0,0,1,0,1,0,0,2,0,0,2,0
3,-7,1,1585009049,1,0,0,1,0,1,0,0,3,0,0,3,0
4,-7,1,1585009056,1,0,0,1,0,1,0,0,4,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7441,0,1591339821,667,3,22,4,101,2,5,157,4,5,157,1396,217
2525952,7441,1,1597971999,274,5,3,4,11,0,3,124,0,3,124,5041,809
2525953,7441,1,1597972010,274,5,3,4,11,2,3,124,1,3,124,5042,809
2525954,7441,1,1597972056,274,5,3,4,11,2,3,124,2,3,124,5043,809


In [13]:
df["user_correct_answer"] = (
    df.groupby("userID")["answerCode"]
    .transform(lambda x: x.cumsum().shift(1))
    .fillna(0)
)

df["user_total_answer"] = df.groupby("userID")["answerCode"].cumcount()
df["user_acc"] = (df["user_correct_answer"] / df["user_total_answer"]).fillna(0)

df

,userID,answerCode,Timestamp,KnowledgeTag,month,hour,week,week_hour,elapsed,assessmentItemID0,assessmentItemID1,assessmentItemID2,testId0,testId1,assessmentItemID,testId,user_correct_answer,user_total_answer,user_acc
0,-7,1,1585009031,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0,0.000000
1,-7,1,1585009034,1,0,0,1,0,1,0,0,1,0,0,1,0,1.0,1,1.000000
2,-7,1,1585009042,1,0,0,1,0,1,0,0,2,0,0,2,0,2.0,2,1.000000
3,-7,1,1585009049,1,0,0,1,0,1,0,0,3,0,0,3,0,3.0,3,1.000000
4,-7,1,1585009056,1,0,0,1,0,1,0,0,4,0,0,4,0,4.0,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7441,0,1591339821,667,3,22,4,101,2,5,157,4,5,157,1396,217,1.0,4,0.250000
2525952,7441,1,1597971999,274,5,3,4,11,0,3,124,0,3,124,5041,809,1.0,5,0.200000
2525953,7441,1,1597972010,274,5,3,4,11,2,3,124,1,3,124,5042,809,2.0,6,0.333333
2525954,7441,1,1597972056,274,5,3,4,11,2,3,124,2,3,124,5043,809,3.0,7,0.428571


In [28]:
train = pd.read_csv('../../data/train_test_data.csv')
train['problem_number'] = train.assessmentItemID.str[-2:]
train['problem_number'] = train.problem_number.astype(np.int16)
train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,problem_number
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5
...,...,...,...,...,...,...,...
2525951,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132,6
2525952,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,1
2525953,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,2
2525954,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,3


In [30]:
f = lambda x: len(set(x))
test = train.groupby(["testId"]).agg({"problem_number": "max", "KnowledgeTag": f})
test.reset_index(inplace=True)

test.columns = ["merge", "problem_count", "tag_count"]
#test
df['merge'] = train['testId']
df = pd.merge(df, test, on="merge", how="left")

In [31]:
df["problem_position"] = df["problem_number"] / df["problem_count"]
df["solve_order"] = df.groupby(["userID", "testId"]).cumcount()
df["solve_order"] = (
    df["solve_order"]
    - df["problem_count"] * (df["solve_order"] > df["problem_count"]).apply(int)
    + 1
)

,userID,answerCode,Timestamp,KnowledgeTag,month,hour,week,week_hour,elapsed,assessmentItemID0,...,testId0,testId1,assessmentItemID,testId,user_correct_answer,user_total_answer,user_acc,merge,problem_count,tag_count
0,-7,1,1585009031,0,0,0,1,0,0,0,...,0,0,0,0,0.0,0,0.000000,A060000001,7,2
1,-7,1,1585009034,1,0,0,1,0,1,0,...,0,0,1,0,1.0,1,1.000000,A060000001,7,2
2,-7,1,1585009042,1,0,0,1,0,1,0,...,0,0,2,0,2.0,2,1.000000,A060000001,7,2
3,-7,1,1585009049,1,0,0,1,0,1,0,...,0,0,3,0,3.0,3,1.000000,A060000001,7,2
4,-7,1,1585009056,1,0,0,1,0,1,0,...,0,0,4,0,4.0,4,1.000000,A060000001,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7441,0,1591339821,667,3,22,4,101,2,5,...,5,157,1396,217,1.0,4,0.250000,A040000197,6,1
2525952,7441,1,1597971999,274,5,3,4,11,0,3,...,3,124,5041,809,1.0,5,0.200000,A040000130,5,2
2525953,7441,1,1597972010,274,5,3,4,11,2,3,...,3,124,5042,809,2.0,6,0.333333,A040000130,5,2
2525954,7441,1,1597972056,274,5,3,4,11,2,3,...,3,124,5043,809,3.0,7,0.428571,A040000130,5,2
